# Replicate Total COVID Cases Chart

In the first section, we get the data for creating a linechart using [Altair](https://altair-viz.github.io/) ([vega-lite](https://vega.github.io/vega-lite/) for Python). On the second section, we pre-process use it as a DataWrapper input. Caveat, Altair uses the pandas long format to find the data points per category; DataWrapper prefers the wide format.

In [1]:
import math
import pandas as pd
import numpy as np

# only used for 
import altair as alt
from vega_datasets import data
from altair import datum
from altair import Scale
from altair import *
%matplotlib inline
# Optional in Jupyter Notebook: requires an up-to-date vega nbextension.
#alt.renderers.enable('notebook')
alt.renderers.enable('default')

RendererRegistry.enable('default')

## 1. Chart with Altair

In [2]:
# Total tests per day for whole Peru, first tab from
# https://github.com/jincio/COVID_19_PERU/blob/master/reportes_minsa.xlsx
df = pd.read_csv('data/minsa_total.csv')

# Parse "Month-day" format to date column
date = df['Dia'].apply(lambda x: x + "-2020")
df = df.drop(['Dia', 'Hora'], axis=1)
df['date'] = pd.to_datetime(date, format="%d-%b-%Y")
# Number of days since first report. This works because the data is already
# ordered by date (can sort as safety check)
df['days'] = df.index

# clean data for positive cases from serological tests
df['RapidasPositivos'] = df['RapidasPositivos'].fillna(0)
# total cases per day
df['total_cases'] = (df['Positivos'] + df['RapidasPositivos']).astype('int32')

# Calculate the log values for Altair
df['log_total_cases'] = np.log10(df['total_cases'])
# df['log_positive_molecular'] = np.log10(df['Positivos'])
# df['log_positive_serological'] = np.log10(df['RapidasPositivos'])
# df.loc[df['log_positive_serological'] == -1 * math.inf, 'log_positive_serological']=0

In [3]:
df.head()

,Total_Pruebas,Descartados,Positivos,Nuevos_Positivos,TasaPositivos,Pruebas_dia,Recuperados,Fallecidos,Hospitalizados,Hospitalizados_UCI,Hospitalizados_ventilador,PruebasRapidas,RapidasPositivos,Pruebas_diaPR,PR_nuevos,PM_PR,date,days,total_cases,log_total_cases
0,155,154,1,1,0.645161,155,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2020-03-06,0,1,0.000000
1,219,213,6,5,2.739726,64,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2020-03-07,1,6,0.778151
2,250,243,7,1,2.800000,31,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2020-03-08,2,7,0.845098
3,318,309,9,2,2.830189,68,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2020-03-09,3,9,0.954243
4,346,335,11,2,3.179191,28,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2020-03-10,4,11,1.041393


## Doubling times
Calculate doubling rates based on the initial population, number of days to double the number of infected people and the number of days passed so far since the first report.
https://blog.datawrapper.de/weekly-chart-coronavirus-doublingtimes/

In [4]:
def get_doubling_numbers(number_days_to_double, initial_population, days_passed, trend_label):
    x = np.arange(days_passed)
    f = lambda t: initial_population * math.pow(math.pow(2, 1/number_days_to_double), t)
    vf = np.vectorize(f)
    
    y = vf(x)
    df_every_x = pd.DataFrame({ 
        'days': x,
        'total_cases': y,
        'log_total': np.log10(y), 
        'trend': [trend_label] * (days_passed)
    })
        
    return df_every_x

initial_population = 1
days_passed = df.shape[0]

df_every_day = get_doubling_numbers(1, initial_population, days_passed, 'cada dia')
df_every_two_days = get_doubling_numbers(2, initial_population, days_passed, 'cada dos dias')
df_every_three_days = get_doubling_numbers(3, initial_population, days_passed, 'cada tres dias')
df_every_cuatro_days = get_doubling_numbers(4, initial_population, days_passed, 'cada cuatro dias')
df_every_week = get_doubling_numbers(7, initial_population, days_passed, 'cada semana')

trends = pd.concat([df_every_day, df_every_two_days, df_every_three_days, df_every_cuatro_days, df_every_week])

## Sample Chart: Total Cases

In [5]:
ylim = 5.5

lines = alt.Chart(df).mark_line().encode(
    alt.X('days:Q', axis=alt.Axis(title="Dias desde primer reporte")),
    alt.Y('log_total_cases:Q', axis=alt.Axis(title="Casos Totals (log)"), scale=alt.Scale(domain=(0, ylim), clamp=True))
)

# Adding on mouseover interaction
nearest = alt.selection(type='single', nearest=True, on='mouseover', fields=['days'], empty='none')

selectors = alt.Chart(df).mark_point().encode(
    x='days:Q',
    opacity=alt.value(0),
).add_selection(nearest)

points = lines.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

rules = alt.Chart(df).mark_rule(color='gray').encode(
    x='days:Q',
).transform_filter(nearest)

points = lines.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

text = lines.mark_text(align='left', dx=3, dy=-18, baseline='middle', fontSize=12).encode(
    text=alt.condition(nearest, 'total_cases:Q', alt.value(' '))
)

rules = alt.Chart(df).mark_rule(color='gray').encode(
    x='days:Q',
).transform_filter(nearest)

# Trend lines
trend_lines = alt.Chart(trends).mark_line(strokeDash=[1,4], color="silver").encode(
    alt.X('days:Q'),
    alt.Y('log_total:Q', scale=alt.Scale(domain=(0, ylim), clamp=True)),
    detail='trend'
)

anno_every_day = alt.Chart(trends).mark_text(
    align='right',
    baseline='middle',
    fontSize = 13,
    dx = -8
).encode(
    x='days',
    y='log_total', 
    text=alt.value('... cada dia')
).transform_filter(
    (datum.days == 17) & (datum.trend == 'cada dia')
)

anno_every_two_days = alt.Chart(trends).mark_text(
    align='right',
    baseline='middle',
    fontSize = 13,
    dx = -8
).encode(
    x='days',
    y='log_total', 
    text=alt.value('... cada dos dias')
).transform_filter(
    (datum.days == 35) & (datum.trend == 'cada dos dias')
)

anno_every_three_days = alt.Chart(trends).mark_text(
    align='right',
    baseline='middle',
    fontSize = 13,
    dx = -8
).encode(
    x='days',
    y='log_total', 
    text=alt.value('... cada tres dias')
).transform_filter(
    (datum.days == 52) & (datum.trend == 'cada tres dias')
)

anno_every_four_days = alt.Chart(trends).mark_text(
    align='left',
    baseline='middle',
    fontSize = 13,
    dx = 4
).encode(
    x='days',
    y='log_total', 
    text=alt.value('... cada cuatro dias')
).transform_filter(
    (datum.days == 58) & (datum.trend == 'cada cuatro dias')
)

anno_every_week = alt.Chart(trends).mark_text(
    align='left',
    baseline='middle',
    fontSize = 13,
    dx = 4
).encode(
    x='days',
    y='log_total', 
    text=alt.value('... cada semana')
).transform_filter(
    (datum.days == 56) & (datum.trend == 'cada semana')
)


alt.layer(
    lines, selectors, points, rules, text, trend_lines, 
    anno_every_day, anno_every_two_days, anno_every_three_days, anno_every_four_days, anno_every_week,
).properties(
    width=900, height=600, background="#fff1e5", 
    title={
      "text": ["Total de casos acumulados en el Peru"], 
      "subtitle": ["Grafico en escala logaritmica. Fuente: https://github.com/jincio/COVID_19_PERU"]
    }
)

alt.LayerChart(...)

## 2. Pre-process the data for DataWrapper

As we are interested in the number of cases for the whole country and per individual regions, let's aim for the following columns: *region*, *total_cases*, *days*. We can drop any log calculation as DataWrapper allows to change the column to logaritmic scale if needed. We will start with the long format and convert to wide format at the end.

In [6]:
# I'm repeating the steps from above and discarding any additional column
peru_df = pd.read_csv('data/minsa_total.csv')

date = peru_df['Dia'].apply(lambda x: x + "-2020")
peru_df = peru_df.drop(['Dia', 'Hora'], axis=1)
peru_df['date'] = pd.to_datetime(date, format="%d-%b-%Y")
peru_df['days'] = peru_df.index

peru_df['RapidasPositivos'] = peru_df['RapidasPositivos'].fillna(0)
peru_df['total_cases'] = (peru_df['Positivos'] + peru_df['RapidasPositivos']).astype('int32')
peru_df = peru_df[['total_cases', 'date', 'days']]
# let's add region PERU for these values as we will add later the data per region
peru_df['region'] = ['PERU'] * peru_df.shape[0]
# no need for date
peru_df = peru_df.drop(columns=['date'])

peru_df.head()

,total_cases,days,region
0,1,0,PERU
1,6,1,PERU
2,7,2,PERU
3,9,3,PERU
4,11,4,PERU


### Adjusting the trends dataframe

In [7]:
# reusing the "region" column for trends.
trends_dw = trends.copy()
trends_dw = trends_dw.rename(columns={"trend": "region"})
trends_dw = trends_dw.drop(columns=['log_total'])
trends_dw.head()

,days,total_cases,region
0,0,1.0,cada dia
1,1,2.0,cada dia
2,2,4.0,cada dia
3,3,8.0,cada dia
4,4,16.0,cada dia


### Getting the data per region

In [8]:
# Second tab from Reportes MINSA
# https://github.com/jincio/COVID_19_PERU/blob/master/reportes_minsa.xlsx
orig_df_dep = pd.read_csv('data/casos_departamento.csv')
# Standarize number format as the original column uses multiple formats for numbers
# e.g. 1000 preferred over 1,000
orig_df_dep['Positivos_totales'] = orig_df_dep['Positivos_totales'].str.replace(',', '')
orig_df_dep['Positivos_totales'] = orig_df_dep['Positivos_totales'].astype('int32')

In [9]:
orig_df_dep.head()

,COUNTRY,REGION,Positivos_PCR,Positivos_PR,Positivos_PCR_PR,Positivos_totales,Fallecidos,Total_muestras,Fecha,PositivosImputados_PCR,PositivosImputados_PR,PositivosImputados_PCR_PR,PositivosImputados_totales,Fallecidos_cdc,MuestrasImputadas_totales,Nota
0,PE,LIMA,1,NaN,NaN,1,NaN,NaN,6-Mar,1.0,NaN,NaN,1,NaN,NaN,NaN
1,PE,LIMA,6,NaN,NaN,6,NaN,NaN,7-Mar,6.0,NaN,NaN,6,NaN,NaN,NaN
2,PE,LIMA,7,NaN,NaN,7,NaN,NaN,8-Mar,7.0,NaN,NaN,7,NaN,NaN,NaN
3,PE,LIMA,9,NaN,NaN,9,NaN,NaN,9-Mar,9.0,NaN,NaN,9,NaN,NaN,NaN
4,PE,LIMA,11,NaN,NaN,11,NaN,NaN,10-Mar,11.0,NaN,NaN,11,NaN,NaN,NaN


In [10]:
# check column Positivos_PR on row 1150 as an example of multiple number formatting
orig_df_dep[orig_df_dep['REGION'] == 'LIMA']

,COUNTRY,REGION,Positivos_PCR,Positivos_PR,Positivos_PCR_PR,Positivos_totales,Fallecidos,Total_muestras,Fecha,PositivosImputados_PCR,PositivosImputados_PR,PositivosImputados_PCR_PR,PositivosImputados_totales,Fallecidos_cdc,MuestrasImputadas_totales,Nota
0,PE,LIMA,1,NaN,NaN,1,NaN,NaN,6-Mar,1.0,NaN,NaN,1,NaN,NaN,NaN
1,PE,LIMA,6,NaN,NaN,6,NaN,NaN,7-Mar,6.0,NaN,NaN,6,NaN,NaN,NaN
2,PE,LIMA,7,NaN,NaN,7,NaN,NaN,8-Mar,7.0,NaN,NaN,7,NaN,NaN,NaN
3,PE,LIMA,9,NaN,NaN,9,NaN,NaN,9-Mar,9.0,NaN,NaN,9,NaN,NaN,NaN
4,PE,LIMA,11,NaN,NaN,11,NaN,NaN,10-Mar,11.0,NaN,NaN,11,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,PE,LIMA,12001,17841,NaN,29842,529.0,213745,4-May,12001.0,17841.0,NaN,29842,NaN,213745,NaN
1100,PE,LIMA,"12,294","20,045",NaN,32339,584.0,"224,601",5-May,12294.0,20045.0,NaN,32339,NaN,224601,NaN
1125,PE,LIMA,12935,22364,NaN,35299,613.0,237155,6-May,12935.0,22364.0,NaN,35299,NaN,237155,NaN
1150,PE,LIMA,"13,639","23,967",NaN,37606,636.0,"245,989",7-May,13639.0,23967.0,NaN,37606,NaN,245989,NaN


In [11]:
# Following up with the data preprocessing...

dep_df = orig_df_dep.copy()
# keep relevant columns
dep_df = orig_df_dep[['Positivos_totales', 'Fecha', 'REGION']]
# update date format
date = dep_df['Fecha'].apply(lambda x: x + "-2020")
dep_df['date'] = pd.to_datetime(date, format="%d-%b-%Y")

# temporal dictionary with the initial date of reporting per region
first_day_df = pd.DataFrame(dep_df.groupby('REGION').min()['date'])
first_day_df = first_day_df.to_dict('index')
# create a temporal column for the initial date of reporting
dep_df['init_date'] = [first_day_df[d]['date'] for d in dep_df.get('REGION')]

def calc_days_passed(curr_date, init_date):
    first_date = pd.Timestamp(init_date).to_pydatetime()
    curr_date = pd.Timestamp(curr_date).to_pydatetime()
    return abs(curr_date - first_date).days
fx = np.vectorize(calc_days_passed)    

# calculate the number of days passed since initial reporting per region
dep_df['days'] = fx(dep_df['date'], dep_df['init_date'])

# adjusting columns
dep_df = dep_df.rename(columns={"Positivos_totales": "total_cases", "REGION": "region"})
dep_df = dep_df.drop(columns=['init_date', 'date', 'Fecha'])
dep_df.head()

D:\anaconda3\envs\ds\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\anaconda3\envs\ds\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\anaconda3\envs\ds\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

,total_cases,region,days
0,1,LIMA,0
1,6,LIMA,1
2,7,LIMA,2
3,9,LIMA,3
4,11,LIMA,4


### Merging dataframes and converting to wide format

In [12]:
final_df = pd.concat([peru_df, dep_df, trends_dw])

In [13]:
final_df

,total_cases,days,region
0,1.000000,0,PERU
1,6.000000,1,PERU
2,7.000000,2,PERU
3,9.000000,3,PERU
4,11.000000,4,PERU
...,...,...,...
59,344.550449,59,cada semana
60,380.414538,60,cada semana
61,420.011702,61,cada semana
62,463.730516,62,cada semana


In [14]:
# to wide format
piv_final_df = final_df.pivot(index='days', columns='region', values='total_cases')

In [15]:
piv_final_df.head()

region,AMAZONAS,ANCASH,APURIMAC,AREQUIPA,AYACUCHO,CAJAMARCA,CALLAO,CUSCO,HUANCAVELICA,HUANUCO,...,PUNO,SAN MARTIN,TACNA,TUMBES,UCAYALI,cada cuatro dias,cada dia,cada dos dias,cada semana,cada tres dias
days,,,,,,,,,,,,,,,,,,,,,
0,11.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,...,1.0,1.0,1.0,3.0,12.0,1.000000,1.0,1.000000,1.000000,1.000000
1,10.0,1.0,1.0,2.0,1.0,1.0,3.0,1.0,1.0,2.0,...,1.0,NaN,1.0,3.0,14.0,1.189207,2.0,1.414214,1.104090,1.259921
2,16.0,1.0,1.0,2.0,1.0,1.0,4.0,1.0,1.0,2.0,...,1.0,NaN,2.0,3.0,15.0,1.414214,4.0,2.000000,1.219014,1.587401
3,22.0,2.0,1.0,2.0,1.0,1.0,5.0,1.0,1.0,2.0,...,1.0,NaN,3.0,5.0,17.0,1.681793,8.0,2.828427,1.345900,2.000000
4,19.0,4.0,1.0,2.0,1.0,1.0,5.0,1.0,1.0,2.0,...,1.0,1.0,3.0,6.0,32.0,2.000000,16.0,4.000000,1.485994,2.519842


In [18]:
piv_final_df.to_csv('data/piv_data.csv', index=True)

In [19]:
# It's ok to keep the NaNs, DataWrapper will ignore it
piv_final_df.tail()

region,AMAZONAS,ANCASH,APURIMAC,AREQUIPA,AYACUCHO,CAJAMARCA,CALLAO,CUSCO,HUANCAVELICA,HUANUCO,...,PUNO,SAN MARTIN,TACNA,TUMBES,UCAYALI,cada cuatro dias,cada dia,cada dos dias,cada semana,cada tres dias
days,,,,,,,,,,,,,,,,,,,,,
59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,27554.493735,5.764608e+17,7.592501e+08,344.550449,8.322553e+05
60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,32768.000000,1.152922e+18,1.073742e+09,380.414538,1.048576e+06
61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,38967.938744,2.305843e+18,1.518500e+09,420.011702,1.321123e+06
62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,46340.950012,4.611686e+18,2.147484e+09,463.730516,1.664511e+06
63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,55108.987470,9.223372e+18,3.037000e+09,512.000000,2.097152e+06


Now go to [DataWrapper](https://www.datawrapper.de/) to create the chart. Check Readme.